In [1]:
import glob
from collections import namedtuple

import json
import pandas as pd
from tqdm import tqdm

In [2]:
DATA_DIR = "/home/toor/Desktop/"
experiment = "AA"

In [3]:
data = pd.concat([
    pd.read_json(data_path, lines=True) 
    for data_path 
    in glob.glob(DATA_DIR + '/data/*/data.*')
])

In [4]:
# data["rnd"] = np.random.random(len(data))
data["treatment"] = data["experiments"].map(lambda experiments: experiments[experiment])

data = data[['user', 'track', 'message', 'timestamp', 'time', 'treatment']]

In [6]:
track_meta = pd.read_json("../data/tracks.json", lines=True)
data = pd.merge(data, track_meta[['artist','track']], on="track")

In [7]:
data.shape

(23755873, 7)

In [8]:
data.max()

user                               9999
track                             49999
message                            next
timestamp    2024-11-29 00:56:38.141000
time                                1.0
treatment                            T1
artist                               阿楷
dtype: object

In [9]:
Session = namedtuple("Session", ["timestamp", "track", "time", "artist"])

def sessionize(user_data):
    sessions = []
    session = None
    for _, row in user_data.sort_values("timestamp").iterrows():
        if session is None:
            session = Session(row['timestamp'], [], [], [])

        if (not row['track'] in session.track) and (not row['artist'] in session.artist):
            session.track.append(row['track'])
            session.artist.append(row['artist'])
            session.time.append(row['time'])

        if row["message"] == "last":
            sessions.append(session._asdict())
            session = None
    return sessions

In [10]:
sessions = (
    data
    .groupby(["user", "treatment"])
    .apply(sessionize)
    .explode()
    .apply(pd.Series)
)

/tmp/ipykernel_292197/2354129256.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(sessionize)


In [11]:
prepare_data = []
for (user, treatment), row in tqdm(sessions.iterrows()):
    for track, time in zip(row['track'], row['time']):
        new_record = {
            'user': user,
            'first_track': row['track'][0],
            'track': track,
            'time': time}
        prepare_data.append(new_record)

# with open('../data/prepare_data.json', 'w') as f:
#   json.dump(prepare_date, f)

3175000it [01:15, 41843.84it/s]


In [12]:
df = pd.DataFrame(prepare_data)
df = df[df.first_track != df.track]
df.to_csv('../data/data.csv', index=False)

In [13]:
len(df)

20526321

In [14]:
df.first_track.nunique()

43481